# 📊 Analisi Dati HR: Turnover e Demografia Aziendale

## Guida Completa per Manager HR

**Creato il:** 18 Luglio 2025  
**Destinatari:** Manager HR, Direttori del Personale, Leadership aziendale  
**Livello:** Principiante - Non richiede conoscenze tecniche  

---

### 🎯 **Cosa troverai in questa analisi:**

1. **Demografia dei dipendenti** - Chi sono i nostri dipendenti?
2. **Analisi salariale** - Come sono distribuite le retribuzioni?
3. **Performance aziendale** - Come performano i nostri team?
4. **Turnover e retention** - Quanti dipendenti lasciano l'azienda?
5. **Analisi correlazioni** - Relazioni tra variabili
6. **Raccomandazioni strategiche** - Cosa fare con questi dati?

### 💡 **Come leggere questo notebook:**
- **Celle grigie** = Codice (esegui con Shift+Enter)
- **Testo normale** = Spiegazioni e interpretazioni
- **📈 Grafici** = Visualizzazioni dei dati
- **🔍 Insights** = Conclusioni pratiche per l'HR

---

## 🔧 Preparazione dell'Ambiente

**❗ IMPORTANTE:** Esegui questa cella per prima cosa premendo **Shift+Enter**

In [ ]:
# 🔧 CONTROLLO E INSTALLAZIONE AUTOMATICA LIBRERIE
import subprocess
import sys

def installa_libreria(nome_libreria):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", nome_libreria])
        print(f"✅ {nome_libreria} installata con successo!")
    except subprocess.CalledProcessError:
        print(f"❌ Errore nell'installazione di {nome_libreria}")

# Lista delle librerie necessarie
librerie_necessarie = {
    'pandas': 'pandas',
    'numpy': 'numpy', 
    'matplotlib': 'matplotlib',
    'seaborn': 'seaborn'
}

print("🔍 Controllo librerie necessarie...")
librerie_mancanti = []

for nome_import, nome_pip in librerie_necessarie.items():
    try:
        __import__(nome_import)
        print(f"✅ {nome_import} - OK")
    except ImportError:
        print(f"❌ {nome_import} - MANCANTE")
        librerie_mancanti.append(nome_pip)

if librerie_mancanti:
    print(f"\n🔧 Installazione di {len(librerie_mancanti)} librerie mancanti...")
    for libreria in librerie_mancanti:
        installa_libreria(libreria)
    print("\n🔄 Riavvio dell'importazione...")
else:
    print("\n🎉 Tutte le librerie sono già installate!")

# ===== IMPORTAZIONE DELLE LIBRERIE =====
print("\n📦 Importazione librerie per l'analisi dati...")

try:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from datetime import datetime
    import warnings
    import re
    
    warnings.filterwarnings('ignore')
    
    # Configurazione grafici
    plt.style.use('default')
    sns.set_palette("husl")
    plt.rcParams['figure.figsize'] = (12, 6)
    plt.rcParams['font.size'] = 10
    
    print("✅ Tutte le librerie importate con successo!")
    print("🎯 Versioni installate:")
    print(f"   • Pandas: {pd.__version__}")
    print(f"   • NumPy: {np.__version__}")
    print(f"   • Matplotlib: {plt.matplotlib.__version__}")
    print(f"   • Seaborn: {sns.__version__}")
    print("\n🚀 Ambiente pronto per l'analisi HR!")
    
except ImportError as e:
    print(f"❌ Errore durante l'importazione: {e}")
    print("💡 Prova a riavviare il kernel e rieseguire questa cella.")

## 📂 Caricamento e Pulizia dei Dati HR

**Operazioni automatiche:**
- ✅ Carica il file "hr_data.csv"
- ✅ Corregge date malformate (es. "06/17,1989" → "06/17/1989")
- ✅ Pulisce errori nei dati
- ✅ Calcola età e anzianità

**Per Manager HR:** Concentrati sui risultati!

In [ ]:
def carica_e_pulisci_dati():
    """Carica e pulisce i dati HR in un'unica operazione"""
    
    # Lista dei possibili nomi file
    possibili_file = ['hr_data.csv', 'HR_Data.csv', 'HR Data.csv']
    
    for file_name in possibili_file:
        try:
            print(f"📁 Tentativo di caricamento del file {file_name}...")
            
            # Caricamento con gestione errori
            try:
                df = pd.read_csv(file_name)
            except pd.errors.ParserError:
                print("🔧 Errore parsing CSV, uso parametri robusti...")
                df = pd.read_csv(file_name, on_bad_lines='skip', skipinitialspace=True)
            
            print(f"✅ Dati caricati da {file_name}! Totale: {len(df):,} dipendenti")
            
            # PULIZIA AUTOMATICA
            print("🧹 Pulizia automatica dei dati...")
            
            # Correggi date malformate
            colonne_date = ['DateOfBirth', 'HiringDate', 'TerminationDate']
            for col in colonne_date:
                if col in df.columns:
                    df[col] = df[col].astype(str).str.strip()
                    # Correggi virgole nelle date (problema principale!)
                    df[col] = df[col].str.replace(r'(\\d{2})/(\\d{2}),(\\d{4})', r'\\1/\\2/\\3', regex=True)
                    df[col] = df[col].replace(['nan', ''], None)
            
            # Converti date
            for col in colonne_date:
                if col in df.columns:
                    df[col] = pd.to_datetime(df[col], errors='coerce')
            
            # Pulisci campo Gender
            if 'Gender' in df.columns:
                df['Gender'] = df['Gender'].str.strip()
                df['Gender'] = df['Gender'].replace({'M ': 'M'})
            
            # Calcola età e anzianità (con gestione NaN)
            oggi = pd.Timestamp('2025-07-18')
            if 'DateOfBirth' in df.columns:
                # Calcola età solo per date valide
                eta_days = (oggi - df['DateOfBirth']).dt.days
                df['Eta'] = (eta_days / 365.25).round().astype('Int64')  # Int64 supporta NaN
                df['Eta'] = df['Eta'].clip(lower=0, upper=100)
            
            if 'HiringDate' in df.columns:
                # Calcola anni di servizio solo per date valide
                servizio_days = (oggi - df['HiringDate']).dt.days
                df['AnniServizio'] = (servizio_days / 365.25).round().astype('Int64')  # Int64 supporta NaN
                df['AnniServizio'] = df['AnniServizio'].clip(lower=0, upper=50)
                df['AnnoAssunzione'] = df['HiringDate'].dt.year
            
            # Pulisci salari
            if 'Salary' in df.columns:
                df = df[df['Salary'] > 0]
                df = df[df['Salary'] < 1000000]
            
            # Rimuovi righe con dati critici mancanti
            if 'EmployeeName' in df.columns:
                df = df.dropna(subset=['EmployeeName'])
            
            print(f"✅ Pulizia completata! Dati finali: {len(df):,} dipendenti")
            if 'AnnoAssunzione' in df.columns:
                print(f"📅 Periodo: {df['AnnoAssunzione'].min()} - {df['AnnoAssunzione'].max()}")
            
            return df
            
        except FileNotFoundError:
            print(f"⚠️  File '{file_name}' non trovato.")
            continue
        except Exception as e:
            print(f"❌ Errore con {file_name}: {e}")
            continue
    
    print("❌ Nessun file HR trovato. Assicurati che 'hr_data.csv' sia presente.")
    return None

# Carica e pulisci i dati
print("🔄 Caricamento e pulizia dati HR...")
df = carica_e_pulisci_dati()

if df is not None:
    print("\n📋 Anteprima dati puliti:")
    colonne_principali = ['EmployeeName', 'Department', 'Salary', 'EmploymentStatus']
    if 'Eta' in df.columns:
        colonne_principali.append('Eta')
    if 'AnniServizio' in df.columns:
        colonne_principali.append('AnniServizio')
    
    colonne_esistenti = [col for col in colonne_principali if col in df.columns]
    print(df[colonne_esistenti].head())
    
    print("\n📊 Riepilogo dataset:")
    print(f"- Dipendenti totali: {len(df):,}")
    print(f"- Colonne disponibili: {len(df.columns)}")
    if 'EmploymentStatus' in df.columns:
        print(f"- Dipendenti attivi: {len(df[df['EmploymentStatus'] == 'Active'])}")
    if 'Salary' in df.columns:
        print(f"- Salario medio: ${df['Salary'].mean():,.0f}")
    if 'Eta' in df.columns:
        print(f"- Età media: {df['Eta'].mean():.1f} anni")
else:
    print("❌ Impossibile procedere senza dati.")